<a href="https://colab.research.google.com/github/niapollard/TEST/blob/main/Table%203_Varying%20Noise%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q numpy==1.23.5 jarvis-tools qiskit==0.43.1 qiskit_nature==0.6.2 pyscf qiskit-aer==0.12.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.6 MB/s eta 0:00:0

Restart the session to succesfully run cells

In [ ]:
import numpy as np

# Importing Qiskit and Jarvis libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from pyscf import gto, scf, tools

<ipython-input-5-ccff2813ef8f>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-5-ccff2813ef8f>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver


In [ ]:
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeSherbrooke, FakeManhattan, FakeTokyo, FakeToronto, FakeMontreal

In [ ]:
# # fake providers contain data from real IBM Quantum devices stored in Qiskit,
# # and are useful for extracting realistic noise models.
# device = FakeSherbrooke()

# coupling_map = device.coupling_map
# noise_model = NoiseModel.from_backend(device)

# print(noise_model)

In [ ]:
#Build the molecule
molecule = Molecule(
    geometry=[['Al',[14.46097199, -11.1250000, 0.00000000]],
              ['Al',[17.61662800, -11.1250000, 0.00000000]]],
    charge=0,
    multiplicity=1)

In [ ]:
fake_devices = [FakeSherbrooke(),FakeManhattan(),FakeToronto(),FakeTokyo(),FakeMontreal()]

def get_qubit_op(
    molecule,
    basis='sto3g',
    functional='lda',
    method= MethodType.RKS,
    driver_type= ElectronicStructureDriverType.PYSCF,
    converter= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',backend='',qubit_op='',seed='', reps=1, noise_model='', coupling_map=''):
  seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  print('backend',backend)
  print('seed',seed)

  qi = QuantumInstance(backend, noise_model=noise_model, coupling_map=coupling_map, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['sto3g']
method=[MethodType.RKS]
optimizer=[SLSQP(maxiter=1000)]
functionals=['lda']
backend=[Aer.get_backend('qasm_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          #for n in converter:
            for o in functionals:
              for p in backend:
                for reps in reps_list:
                  for device in fake_devices:
                   print(i,j,k,m,o,p,reps,device)

                   noise_model = NoiseModel.from_backend(device)

                   try:
                      coupling_map = device.configuration().coupling_map
                   except AttributeError:
                      if hasattr(device, "coupling_map"):
                          coupling_map = device.coupling_map  # Directly retrieve

                   res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k)
                   res=get_energy(optimizer=m,backend=p,qubit_op=res1['qubit_op'],seed=42,reps=reps,noise_model=noise_model,coupling_map=coupling_map)
                   print(res['eigenvalue'])

                   #GroundStateEigensolver
                   solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                   result=solver.solve(res1['problem_reduced'])
                   print(result)

                  #  #numpy
                  #  print("Numpy Energy Result")
                  #  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  #  result=solver.solve(res1['problem_reduced'])
                  #  print(result)

                   mem[i,j,k,m,o,p,reps]= {
                       'eigenvalue': res['eigenvalue'],
                       'vqe': res['vqe'],
                       'qi': res['qi'],
                       'converter': res1['converter'],
                       'problem_reduced': res1['problem_reduced']
                      }

#print(mem)

Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7cc8a2828790> lda qasm_simulator 1 <qiskit.providers.fake_provider.backends.sherbrooke.fake_sherbrooke.FakeSherbrooke object at 0x7cc8a29dbc70>


<ipython-input-8-e86676056b23>:106: NatureDeprecationWarning: The qiskit_nature.problems.second_quantization.electronic.ElectronicStructureProblem class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.problems.ElectronicStructureProblem class.
  res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k)
<ipython-input-8-e86676056b23>:19: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


backend qasm_simulator
seed 42


<ipython-input-8-e86676056b23>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(backend, noise_model=noise_model, coupling_map=coupling_map, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-8-e86676056b23>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(3.705078125+0j)


...

(3.4638671875000004+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -505.532802007965
  - computed part:      -1.385381546142
  - ActiveSpaceTransformer extracted energy part: -504.147420461823
~ Nuclear repulsion energy (Hartree): 28.339891408341
> Total ground state energy (Hartree): -477.192910599624
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.559 S: 0.660 S^2: 1.096 M: 0.099
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [788.03242329  -546.60328153  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [743.9848947  -515.9073408  -0.02097028]
    - computed part:      [77.18822885  -53.39687181  -0.02097028]
    - ActiveSpaceTransformer extracted energy part: [666.79666586  -462.51046899  0.0]
  > Dipole moment (a.u.): [44.04752859  -30.69594073  0.02097028]  Total: 53.68822955
                 (debye): [111.95763977  -78.02129166  0.05330112]  Total: 136.4618551
 
Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP obj

<ipython-input-8-e86676056b23>:19: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


backend qasm_simulator
seed 42


<ipython-input-8-e86676056b23>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(backend, noise_model=noise_model, coupling_map=coupling_map, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-8-e86676056b23>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(3.703125+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -505.543062051829
  - computed part:      -1.395641590006
  - ActiveSpaceTransformer extracted energy part: -504.147420461823
~ Nuclear repulsion energy (Hartree): 28.339891408341
> Total ground state energy (Hartree): -477.203170643487
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.572 S: 0.693 S^2: 1.173 M: 0.074
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [788.03242329  -546.60328153  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [745.44532188  -517.67251333  -0.02912538]
    - computed part:      [78.64865602  -55.16204435  -0.02912538]
    - ActiveSpaceTransformer extracted energy part: [666.79666586  -462.51046899  0.0]
  > Dipole moment (a.u.): [42.58710141  -28.9307682  0.02912538]  Total: 51.4844773
                 (debye): [108.2456045  -73.53467102  0.07402933]  Total: 130.86047615
 
